# ENSEMBLE MODEL TABLE OF CONTENT
- **Uploader**
- **Data Prep**
- **Categorical Data Prep and Modeling**
  - ADHD Model - Undersampled GBDT
  - Sex_F Model - Oversampled GBDT
- **Quantitative Data Prep and Modeling**
- **FCM Data Prep and Modeling**
- **Emsemble Modeling - Stacking Method**
- **Kaggle Submission Maker**

*Maya: I am up to getting best models for Cat rn, if anyone wants to find and input the best models along with their dataprep (dataset modifications) that would be great!*

*Kayla: I updated the categorical data prep and modeling bc we had an updated file with the fixed code after Preston pointed out our sampling mix up. I have also added the best models for quantitative data (even though I get different results than your separate colab for optuna ?)*

In [ ]:
#@title uploader
file_id = "1VpSCc_ms2dj59bMAAOqCu0OI0jjb-8Hq" #@param {type:"string"}
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# PyDrive reference:
# https://googledrive.github.io/PyDrive/docs/build/html/index.html


from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

# Replace the assignment below with your file ID
# to download a different file.
#
# A file ID looks like: 1gLBqEWEBQDYbKCDigHnUXNTkzl-OslSO

import io
from googleapiclient.http import MediaIoBaseDownload

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, done = downloader.next_chunk()

fileId = drive.CreateFile({'id': file_id }) #DRIVE_FILE_ID is file id example: 1iytA1n2z4go3uVCwE_vIKouTKyIDjEq
print(fileId['title'])
fileId.GetContentFile(fileId['title'])  # Save Drive file as a local file

!unzip {fileId['title']}

widsdatathon2025.zip
Archive:  widsdatathon2025.zip
  inflating: Data Dictionary.xlsx    
  inflating: SAMPLE_SUBMISSION.xlsx  
  inflating: TEST/TEST_CATEGORICAL.xlsx  
  inflating: TEST/TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv  
  inflating: TEST/TEST_QUANTITATIVE_METADATA.xlsx  
  inflating: TRAIN/TRAINING_SOLUTIONS.xlsx  
  inflating: TRAIN/TRAIN_CATEGORICAL_METADATA.xlsx  
  inflating: TRAIN/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES.csv  
  inflating: TRAIN/TRAIN_QUANTITATIVE_METADATA.xlsx  


## Data Prep

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

#!pip install scikit-learn
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
#all datasets
train_quantitative = pd.read_excel('TRAIN/TRAIN_QUANTITATIVE_METADATA.xlsx')
train_answers = pd.read_excel('TRAIN/TRAINING_SOLUTIONS.xlsx')
train_categorical = pd.read_excel("TRAIN/TRAIN_CATEGORICAL_METADATA.xlsx")
train_solutions = pd.read_excel("TRAIN/TRAINING_SOLUTIONS.xlsx")
fcm = pd.read_csv("TRAIN/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES.csv")
fcm_solutions = pd.read_excel("TRAIN/TRAINING_SOLUTIONS.xlsx")
graph_fcm = fcm.merge(fcm_solutions, on="participant_id")

In [ ]:
#loading in datasets (I have it stored a different way)
train_categorical = pd.read_excel("/content/TRAIN_CATEGORICAL_METADATA_new.xlsx")
train_solutions = pd.read_excel("/content/TRAINING_SOLUTIONS.xlsx")
train_quantitative = pd.read_excel("/content/TRAIN_QUANTITATIVE_METADATA_new.xlsx")
train_fcm = pd.read_csv("/content/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv.crdownload")

## Categorical Data Prep + Modeling
### ADHD - Oversampling gradient boosted trees (gave worse accuracy?)


In [ ]:
#drop id
for feature in train_categorical.drop(columns = ['participant_id']).columns:
  print(train_categorical[feature].value_counts())
  print("\n")

Basic_Demos_Enroll_Year
2018    415
2019    312
2017    254
2016    191
2015     36
2020      5
Name: count, dtype: int64


Basic_Demos_Study_Site
1    652
3    430
4    120
2     11
Name: count, dtype: int64


PreInt_Demos_Fam_Child_Ethnicity
0.0    777
1.0    296
2.0     77
3.0     20
Name: count, dtype: int64


PreInt_Demos_Fam_Child_Race
0.0     573
8.0     195
1.0     181
2.0     128
3.0      30
9.0      23
10.0     11
4.0      10
11.0      6
7.0       2
Name: count, dtype: int64


MRI_Track_Scan_Location
2.0    532
3.0    463
1.0    179
4.0     36
Name: count, dtype: int64


Barratt_Barratt_P1_Edu
21.0    470
18.0    421
15.0    162
12.0     97
9.0      28
6.0      15
3.0       5
Name: count, dtype: int64


Barratt_Barratt_P1_Occ
0.0     286
35.0    219
45.0    187
30.0    154
40.0    154
25.0     79
15.0     35
5.0      31
20.0     31
10.0      6
Name: count, dtype: int64


Barratt_Barratt_P2_Edu
21.0    323
18.0    301
15.0    166
12.0    162
9.0      44
6.0      14
3.0       5

In [ ]:
df = pd.merge(train_categorical, train_solutions, on='participant_id')

participant_ids = df['participant_id']

In [ ]:
#fill missing NA values
df.fillna({'PreInt_Demos_Fam_Child_Ethnicity': 3}, inplace=True)
df.fillna({'PreInt_Demos_Fam_Child_Race': 10}, inplace=True)
df.dropna(subset=['MRI_Track_Scan_Location', 'Barratt_Barratt_P1_Edu', 'Barratt_Barratt_P1_Occ', 'Barratt_Barratt_P2_Edu', 'Barratt_Barratt_P2_Occ'  ], inplace=True)


In [ ]:
nan_count = np.sum(df.isnull(), axis=0)
nan_count

,0
participant_id,0
Basic_Demos_Enroll_Year,0
Basic_Demos_Study_Site,0
PreInt_Demos_Fam_Child_Ethnicity,0
PreInt_Demos_Fam_Child_Race,0
MRI_Track_Scan_Location,0
Barratt_Barratt_P1_Edu,0
Barratt_Barratt_P1_Occ,0
Barratt_Barratt_P2_Edu,0
Barratt_Barratt_P2_Occ,0


In [ ]:
X = df.drop(columns=['Sex_F', 'participant_id'])
y = df['Sex_F']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rus = RandomUnderSampler(sampling_strategy="auto", random_state=42)
X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)

Running grid search got:

{'learning_rate': 0.01, 'max_depth': 3, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200} Validation accuracy: 0.7366255144032922

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

X_train_ADHD = X_train_resampled.drop(columns=['ADHD_Outcome'])
y_train_ADHD = X_train_resampled['ADHD_Outcome']
x_test_ADHD = X_test.drop(columns=['ADHD_Outcome'])
y_test_ADHD = X_test['ADHD_Outcome']

# Initialize GBDT model with best parameters
gbdt_cat_ADHD = GradientBoostingClassifier(
    learning_rate=0.01,
    max_depth=3,
    min_samples_leaf=2,
    min_samples_split=2,
    n_estimators=200,
    random_state=42
)

# Train the model
gbdt_cat_ADHD.fit(X_train_ADHD, y_train_ADHD)

# Evaluate on validation set
y_pred = gbdt_cat_ADHD.predict(x_test_ADHD)

# Accuracy
accuracy = accuracy_score(y_test_ADHD, y_pred)
print(f"Validation Accuracy: {accuracy:.4f}")

# Weighted F1 Score
f1 = f1_score(y_test_ADHD, y_pred, average='weighted')
print(f"Weighted F1 Score: {f1:.4f}")

Validation Accuracy: 0.6856
Weighted F1 Score: 0.6195


### Sex_f - hybrid gradient boosting trees

In [ ]:
df = pd.merge(train_categorical, train_solutions, on='participant_id')

participant_ids = df['participant_id']

In [ ]:
#fill missing NA values
df.fillna({'PreInt_Demos_Fam_Child_Ethnicity': 3}, inplace=True)
df.fillna({'PreInt_Demos_Fam_Child_Race': 10}, inplace=True)
df.dropna(subset=['MRI_Track_Scan_Location', 'Barratt_Barratt_P1_Edu', 'Barratt_Barratt_P1_Occ', 'Barratt_Barratt_P2_Edu', 'Barratt_Barratt_P2_Occ'  ], inplace=True)

In [ ]:
X = df.drop(columns=['Sex_F', 'ADHD_Outcome', 'participant_id']) #drop ADHD
y = df['Sex_F']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

smote = SMOTE(sampling_strategy="auto", random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

Best parameters: {'learning_rate': 0.001, 'max_depth': 7, 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 500} Validation accuracy: 0.63

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score

# Initialize Gradient Boosting Classifier with best parameters
gbdt_cat_female = GradientBoostingClassifier(
    learning_rate=0.001,
    max_depth=7,
    min_samples_leaf=1,
    min_samples_split=4,
    n_estimators=500,
    random_state=42
)

# Train the model
gbdt_cat_female.fit(X_train_resampled, y_train_resampled)

# Predict on test set
y_pred = gbdt_cat_female.predict(X_test)

# Evaluate performance
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

# Print results
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Weighted F1 Score: {f1:.4f}")


Test Accuracy: 0.6392
Weighted F1 Score: 0.6239


##Quantitative Data Prep


### ADHD - Gradient Boosted DT

{'learning_rate': 0.01, 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}

In [ ]:
# Combine the quantitative data with the ADHD_Outcome labels
train_data = pd.merge(train_quantitative, train_solutions, on='participant_id')

In [ ]:
train_data.fillna({'EHQ_EHQ_Total':train_data['EHQ_EHQ_Total'].mean()}, inplace = True)
train_data.fillna({'ColorVision_CV_Score':train_data['ColorVision_CV_Score'].mean()}, inplace = True)
train_data.fillna({'APQ_P_APQ_P_CP':train_data['APQ_P_APQ_P_CP'].mean()}, inplace = True)
train_data.fillna({'APQ_P_APQ_P_ID':train_data['APQ_P_APQ_P_ID'].mean()}, inplace = True)
train_data.fillna({'APQ_P_APQ_P_INV':train_data['APQ_P_APQ_P_INV'].mean()}, inplace = True)
train_data.fillna({'APQ_P_APQ_P_OPD':train_data['APQ_P_APQ_P_OPD'].mean()}, inplace = True)
train_data.fillna({'APQ_P_APQ_P_PM':train_data['APQ_P_APQ_P_PM'].mean()}, inplace = True)
train_data.fillna({'APQ_P_APQ_P_PP':train_data['APQ_P_APQ_P_PP'].mean()}, inplace = True)

train_data.fillna({'SDQ_SDQ_Conduct_Problems':train_data['SDQ_SDQ_Conduct_Problems'].mean()}, inplace = True)
train_data.fillna({'SDQ_SDQ_Difficulties_Total':train_data['SDQ_SDQ_Difficulties_Total'].mean()}, inplace = True)
train_data.fillna({'SDQ_SDQ_Emotional_Problems':train_data['SDQ_SDQ_Emotional_Problems'].mean()}, inplace = True)
train_data.fillna({'SDQ_SDQ_Externalizing':train_data['SDQ_SDQ_Externalizing'].mean()}, inplace = True)
train_data.fillna({'SDQ_SDQ_Generating_Impact':train_data['SDQ_SDQ_Generating_Impact'].mean()}, inplace = True)
train_data.fillna({'SDQ_SDQ_Hyperactivity':train_data['SDQ_SDQ_Hyperactivity'].mean()}, inplace = True)
train_data.fillna({'SDQ_SDQ_Internalizing':train_data['SDQ_SDQ_Internalizing'].mean()}, inplace = True)
train_data.fillna({'SDQ_SDQ_Peer_Problems':train_data['SDQ_SDQ_Peer_Problems'].mean()}, inplace = True)
train_data.fillna({'SDQ_SDQ_Prosocial':train_data['SDQ_SDQ_Prosocial'].mean()}, inplace = True)
train_data.fillna({'MRI_Track_Age_at_Scan':train_data['MRI_Track_Age_at_Scan'].mean()}, inplace = True)

In [ ]:
nan_count = np.sum(train_data.isnull(), axis=0)
nan_count

,0
participant_id,0
EHQ_EHQ_Total,0
ColorVision_CV_Score,0
APQ_P_APQ_P_CP,0
APQ_P_APQ_P_ID,0
APQ_P_APQ_P_INV,0
APQ_P_APQ_P_OPD,0
APQ_P_APQ_P_PM,0
APQ_P_APQ_P_PP,0
SDQ_SDQ_Conduct_Problems,0


In [ ]:
# Separate features (X) and target variable (y)
X = train_data.drop(['participant_id', 'Sex_F', 'ADHD_Outcome', 'MRI_Track_Age_at_Scan'], axis=1)
y = train_data['ADHD_Outcome']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale numerical features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize and train the Gradient Boosting Classifier
gbdt_quant_ADHD = GradientBoostingClassifier(
    learning_rate=0.01,
    max_depth=3,
    min_samples_leaf=1,
    min_samples_split=2,
    n_estimators=100,
    random_state=42
)

gbdt_quant_ADHD.fit(X_train, y_train)

# Make predictions on the test set
y_pred = gbdt_quant_ADHD.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Gradient Boosting Accuracy: {accuracy:.4f}')

f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Weighted F1 Score: {f1:.4f}")

Gradient Boosting Accuracy: 0.8025
Weighted F1 Score: 0.7837


### Sex_f - Optuna Model

'n_estimators': 150, 'learning_rate': 0.08374413872051761, 'max_depth': 4


In [ ]:
pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 13.6 MB/s eta 0:00:00


In [ ]:
# most code from geeks for geeks
import optuna
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler


# this code is taken from above ################
# Combine the quantitative data with the ADHD_Outcome labels
#train_data = pd.merge(train_quantitative, train_solutions, on='participant_id')

# Separate features (X) and target variable (y)
X = train_data.drop(['participant_id', 'Sex_F', 'ADHD_Outcome', 'MRI_Track_Age_at_Scan'], axis=1)
y = train_data['Sex_F']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale numerical features
scaler2 = StandardScaler()
X_train = scaler2.fit_transform(X_train)
X_test = scaler2.transform(X_test)
###################################################

def objective(trial):
    param_space = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 250, step=50),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'max_depth': trial.suggest_int('max_depth', 3, 7),
    }

    gb_model = GradientBoostingClassifier(**param_space, validation_fraction=0.1, n_iter_no_change=5, random_state=42)

    gb_model.fit(X_train, y_train)

    y_pred = gb_model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)

    return 1.0 - accuracy

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

best_params_optuna = study.best_params
best_model_optuna = GradientBoostingClassifier(**best_params_optuna, validation_fraction=0.1, n_iter_no_change=5, random_state=42)
best_model_optuna.fit(X_train, y_train)

y_pred_best_optuna = best_model_optuna.predict(X_test)

accuracy_best_optuna = accuracy_score(y_test, y_pred_best_optuna)

print(f"Best Model Accuracy (Optuna): {accuracy_best_optuna}")


[I 2025-03-22 17:23:59,492] A new study created in memory with name: no-name-941adf60-4fe4-42d6-96f4-93b8de3caa1b
[I 2025-03-22 17:24:00,620] Trial 0 finished with value: 0.30452674897119336 and parameters: {'n_estimators': 200, 'learning_rate': 0.014295091654364121, 'max_depth': 6}. Best is trial 0 with value: 0.30452674897119336.
[I 2025-03-22 17:24:00,789] Trial 1 finished with value: 0.308641975308642 and parameters: {'n_estimators': 100, 'learning_rate': 0.08072519517313015, 'max_depth': 3}. Best is trial 0 with value: 0.30452674897119336.
[I 2025-03-22 17:24:01,375] Trial 2 finished with value: 0.308641975308642 and parameters: {'n_estimators': 50, 'learning_rate': 0.015034011798774673, 'max_depth': 5}. Best is trial 0 with value: 0.30452674897119336.
[I 2025-03-22 17:24:01,781] Trial 3 finished with value: 0.33333333333333337 and parameters: {'n_estimators': 150, 'learning_rate': 0.1325050329853454, 'max_depth': 7}. Best is trial 0 with value: 0.30452674897119336.
[I 2025-03-22 

Best Model Accuracy (Optuna): 0.7119341563786008


For some reason I am getting that the best parameters are {'n_estimators': 200, 'learning_rate': 0.11596113051499049, 'max_depth': 4} ?

In [ ]:
import optuna
# most code from geeks for geeks



# this code is taken from above ################
# Combine the quantitative data with the ADHD_Outcome labels
#train_data = pd.merge(train_quantitative, train_solutions, on='participant_id')

# Separate features (X) and target variable (y)
X = train_data.drop(['participant_id', 'Sex_F', 'ADHD_Outcome', 'MRI_Track_Age_at_Scan'], axis=1)
y = train_data['Sex_F']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale numerical features
scaler2 = StandardScaler()
X_train = scaler2.fit_transform(X_train)
X_test = scaler2.transform(X_test)
###################################################

gbdt_quant_female = GradientBoostingClassifier(
    learning_rate=0.11596113051499049,
    max_depth=4,
    n_estimators=200,
    validation_fraction=0.1,
    n_iter_no_change=5,
    random_state=42
)
gbdt_quant_female.fit(X_train, y_train)

y_pred_best_optuna = gbdt_quant_female.predict(X_test)

accuracy_best_optuna = accuracy_score(y_test, y_pred_best_optuna)

print(f"Best Model Accuracy (Optuna): {accuracy_best_optuna}")

f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Weighted F1 Score: {f1:.4f}")


Best Model Accuracy (Optuna): 0.7119341563786008
Weighted F1 Score: 0.3270


## FCM Data Prep

In [ ]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.0 MB/s eta 0:00:00


In [ ]:
#reset graph_fcm before running below cell
graph_fcm = fcm.merge(fcm_solutions, on="participant_id")


import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import numpy as np  # Ensure NumPy is imported

# Check if GPU is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Assuming 'graph_fcm' DataFrame is already loaded and preprocessed
# Remove 'Sex_F' column
graph_fcm = graph_fcm.drop('Sex_F', axis=1)

# Prepare data for PyTorch Geometric
connectivity_columns = [col for col in graph_fcm.columns if col not in ['participant_id', 'ADHD_Outcome']]
features = graph_fcm[connectivity_columns].values
labels = graph_fcm['ADHD_Outcome'].values

# Create an adjacency matrix (replace with actual adjacency if available)
num_nodes = features.shape[0]  # Ensure correct shape (number of samples, not features)
adjacency_matrix = np.ones(((int)(num_nodes/2.5), (int)(num_nodes/2.5))) #LITERALLY JUST DIVIDED IN HALF SO ITS NOT FULLY CONNECTED ITS HALF CONNECTED?
np.fill_diagonal(adjacency_matrix, 0)  # No self-loops

# Convert to PyTorch tensors and move to device
data = Data(
    x=torch.tensor(features, dtype=torch.float32).to(device),
    edge_index=torch.tensor(np.array(np.where(adjacency_matrix)), dtype=torch.long).to(device),
    y=torch.tensor(labels, dtype=torch.long).to(device)
)


class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


# Hyperparameters
in_channels = data.num_features
hidden_channels = 64
out_channels = 2  # Two classes: ADHD (1) and no ADHD (0)
learning_rate = 0.01
epochs = 200

# Model, optimizer, and loss function
model_adhd = GCN(in_channels, hidden_channels, out_channels).to(device)
optimizer = torch.optim.Adam(model_adhd.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

# Train the model
model_adhd.train()
for epoch in range(epochs):
    optimizer.zero_grad()
    out = model_adhd(data.x, data.edge_index)
    loss = criterion(out, data.y)
    loss.backward()
    optimizer.step()

    if epoch % 50 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# Evaluation
model_adhd.eval()
_, pred = model_adhd(data.x, data.edge_index).max(dim=1)
correct = float(pred.eq(data.y).sum().item())
acc = correct / len(data.y)
print(f"Accuracy: {acc:.4f}")


In [ ]:
#reset graph_fcm before running below cell
graph_fcm = fcm.merge(fcm_solutions, on="participant_id")


import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import numpy as np  # Ensure NumPy is imported

# Check if GPU is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Assuming 'graph_fcm' DataFrame is already loaded and preprocessed
# Remove 'Sex_F' column
graph_fcm = graph_fcm.drop('ADHD_Outcome', axis=1)

# Prepare data for PyTorch Geometric
connectivity_columns = [col for col in graph_fcm.columns if col not in ['participant_id', 'Sex_F']]
features = graph_fcm[connectivity_columns].values
labels = graph_fcm['Sex_F'].values

# Create an adjacency matrix (replace with actual adjacency if available)
num_nodes = features.shape[0]  # Ensure correct shape (number of samples, not features)
adjacency_matrix = np.ones(((int)(num_nodes/2.5), (int)(num_nodes/2.5)))
np.fill_diagonal(adjacency_matrix, 0)  # No self-loops

# Convert to PyTorch tensors and move to device
data = Data(
    x=torch.tensor(features, dtype=torch.float32).to(device),
    edge_index=torch.tensor(np.array(np.where(adjacency_matrix)), dtype=torch.long).to(device),
    y=torch.tensor(labels, dtype=torch.long).to(device)
)


class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


# Hyperparameters
in_channels = data.num_features
hidden_channels = 64
out_channels = 2  # Two classes: ADHD (1) and no ADHD (0)
learning_rate = 0.01
epochs = 200

# Model, optimizer, and loss function
model_f = GCN(in_channels, hidden_channels, out_channels).to(device)
optimizer = torch.optim.Adam(model_f.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

# Train the model
model_f.train()
for epoch in range(epochs):
    optimizer.zero_grad()
    out = model_f(data.x, data.edge_index)
    loss = criterion(out, data.y)
    loss.backward()
    optimizer.step()

    if epoch % 50 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# Evaluation
model_f.eval()
_, pred = model_f(data.x, data.edge_index).max(dim=1)
correct = float(pred.eq(data.y).sum().item())
acc = correct / len(data.y)
print(f"Accuracy: {acc:.4f}")


#Ensemble Modeling

In [ ]:
df = pd.merge(train_categorical, train_solutions, on='participant_id')
participant_ids = df['participant_id']

df.fillna({'PreInt_Demos_Fam_Child_Ethnicity': 3}, inplace=True)
df.fillna({'PreInt_Demos_Fam_Child_Race': 10}, inplace=True)
df.dropna(subset=['MRI_Track_Scan_Location', 'Barratt_Barratt_P1_Edu', 'Barratt_Barratt_P1_Occ', 'Barratt_Barratt_P2_Edu', 'Barratt_Barratt_P2_Occ'  ], inplace=True)

X = df.drop(columns=['Sex_F', 'participant_id'])
y = df['Sex_F']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rus = RandomUnderSampler(sampling_strategy="auto", random_state=42)
X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)

X_train_ADHD = X_train_resampled.drop(columns=['ADHD_Outcome'])
y_train_ADHD = X_train_resampled['ADHD_Outcome']
x_test_ADHD = X_test.drop(columns=['ADHD_Outcome'])
y_test_ADHD = X_test['ADHD_Outcome']

In [ ]:
X = df.drop(columns=['Sex_F', 'ADHD_Outcome', 'participant_id']) #drop ADHD
y = df['Sex_F']

X_train, X_test_cat_female, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

smote = SMOTE(sampling_strategy="auto", random_state=42)
X_train_resampled_female, y_train_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
train_data = pd.merge(train_quantitative, train_solutions, on='participant_id')
train_data.fillna({'EHQ_EHQ_Total':train_data['EHQ_EHQ_Total'].mean()}, inplace = True)
train_data.fillna({'ColorVision_CV_Score':train_data['ColorVision_CV_Score'].mean()}, inplace = True)
train_data.fillna({'APQ_P_APQ_P_CP':train_data['APQ_P_APQ_P_CP'].mean()}, inplace = True)
train_data.fillna({'APQ_P_APQ_P_ID':train_data['APQ_P_APQ_P_ID'].mean()}, inplace = True)
train_data.fillna({'APQ_P_APQ_P_INV':train_data['APQ_P_APQ_P_INV'].mean()}, inplace = True)
train_data.fillna({'APQ_P_APQ_P_OPD':train_data['APQ_P_APQ_P_OPD'].mean()}, inplace = True)
train_data.fillna({'APQ_P_APQ_P_PM':train_data['APQ_P_APQ_P_PM'].mean()}, inplace = True)
train_data.fillna({'APQ_P_APQ_P_PP':train_data['APQ_P_APQ_P_PP'].mean()}, inplace = True)

train_data.fillna({'SDQ_SDQ_Conduct_Problems':train_data['SDQ_SDQ_Conduct_Problems'].mean()}, inplace = True)
train_data.fillna({'SDQ_SDQ_Difficulties_Total':train_data['SDQ_SDQ_Difficulties_Total'].mean()}, inplace = True)
train_data.fillna({'SDQ_SDQ_Emotional_Problems':train_data['SDQ_SDQ_Emotional_Problems'].mean()}, inplace = True)
train_data.fillna({'SDQ_SDQ_Externalizing':train_data['SDQ_SDQ_Externalizing'].mean()}, inplace = True)
train_data.fillna({'SDQ_SDQ_Generating_Impact':train_data['SDQ_SDQ_Generating_Impact'].mean()}, inplace = True)
train_data.fillna({'SDQ_SDQ_Hyperactivity':train_data['SDQ_SDQ_Hyperactivity'].mean()}, inplace = True)
train_data.fillna({'SDQ_SDQ_Internalizing':train_data['SDQ_SDQ_Internalizing'].mean()}, inplace = True)
train_data.fillna({'SDQ_SDQ_Peer_Problems':train_data['SDQ_SDQ_Peer_Problems'].mean()}, inplace = True)
train_data.fillna({'SDQ_SDQ_Prosocial':train_data['SDQ_SDQ_Prosocial'].mean()}, inplace = True)
train_data.fillna({'MRI_Track_Age_at_Scan':train_data['MRI_Track_Age_at_Scan'].mean()}, inplace = True)

X = train_data.drop(['participant_id', 'Sex_F', 'ADHD_Outcome', 'MRI_Track_Age_at_Scan'], axis=1)
y = train_data['Sex_F']

X_train_quant, X_test_quant, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Get base model predictions
pred_ADHD_1 = gbdt_cat_ADHD.predict_proba(X_train_ADHD)[:, 1]  # Probability of ADHD
pred_ADHD_2 = gbdt_quant_ADHD.predict_proba(X_train_quant)[:, 1]

pred_gender_1 = gbdt_cat_female.predict_proba(X_train_resampled_female)[:, 1]  # Probability of being male or female
pred_gender_2 = gbdt_quant_female.predict_proba(X_train_quant)[:, 1]

# Stack predictions as new features
meta_X_train_ADHD = np.column_stack((pred_ADHD_1, pred_ADHD_2[-534:]))
meta_X_train_female = np.column_stack((pred_gender_1[:970], pred_gender_2)) #I HAD TO CUT OFF THE FIRST MODEL BC THEY HAD DIFFERENT SIZES BUT THERE HAS TO BE A BETTER SOLUTION

# Define meta-targets
meta_y_train_adhd = train_solutions['ADHD_Outcome']  # True ADHD labels
meta_y_train_gender = train_solutions['Sex_F']  # True Gender labels


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(


In [ ]:
print(meta_X_train_female.shape)
print(meta_y_train_gender.shape)

(970, 2)
(1213,)


In [ ]:
from sklearn.linear_model import LogisticRegression

# Meta-model for ADHD
meta_model_adhd = LogisticRegression()
meta_model_adhd.fit(meta_X_train_ADHD, meta_y_train_adhd[-534:])

# Meta-model for Gender
meta_model_gender = LogisticRegression()
meta_model_gender.fit(meta_X_train_female, meta_y_train_gender[:970])

LogisticRegression()

In [ ]:
# Get base model predictions on test data
pred_adhd_1_test = gbdt_cat_ADHD.predict_proba(x_test_ADHD)[:, 1]
pred_adhd_2_test = gbdt_quant_ADHD.predict_proba(X_test_quant)[:, 1]

pred_gender_1_test = gbdt_cat_female.predict_proba(X_test_cat_female)[:, 1]
pred_gender_2_test = gbdt_quant_female.predict_proba(X_test_quant)[:, 1]

# Stack predictions as new features
meta_X_test_ADHD = np.column_stack((pred_adhd_1_test, pred_adhd_2_test[:194]))
meta_X_test_gender = np.column_stack((pred_gender_1_test, pred_gender_2_test[:194]))


# Meta-model predictions
final_pred_adhd = meta_model_adhd.predict(meta_X_test_ADHD)
final_pred_gender = meta_model_gender.predict(meta_X_test_gender)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(


In [ ]:
# Accuracy for ADHD prediction
accuracy_adhd = accuracy_score(y_test_ADHD, final_pred_adhd)
print(f"Accuracy for ADHD prediction: {accuracy_adhd}")

# F1 Score for ADHD prediction
f1_adhd = f1_score(y_test_ADHD, final_pred_adhd, average='weighted')
print(f"F1 Score for ADHD prediction: {f1_adhd}")

# Accuracy for Gender prediction
accuracy_gender = accuracy_score(y_test[:194], final_pred_gender)
print(f"Accuracy for Gender prediction: {accuracy_gender}")

# F1 Score for Gender prediction
f1_gender = f1_score(y_test[:194], final_pred_gender, average='weighted')
print(f"F1 Score for Gender prediction: {f1_gender}")

Accuracy for ADHD prediction: 0.30927835051546393
F1 Score for ADHD prediction: 0.14611575614903807
Accuracy for Gender prediction: 0.7010309278350515
F1 Score for Gender prediction: 0.5778194314276789


In [ ]:
test_categorical = pd.read_excel("/content/TEST_CATEGORICAL.xlsx")
test_quantitative = pd.read_excel("/content/TEST_QUANTITATIVE_METADATA.xlsx")

In [ ]:
X_test_final_quant = test_quantitative.drop(['participant_id', 'MRI_Track_Age_at_Scan'], axis=1)
X_test_final_quant = X_test_final_quant.apply(lambda col: col.fillna(col.mean()), axis=0)

In [ ]:
X_test_final_cat = test_categorical.drop(['participant_id'], axis=1)
X_test_final_cat = X_test_final_cat.apply(lambda col: col.fillna(col.mode()[0]), axis=0)

In [ ]:
# Get base model predictions on test data
pred_adhd_1_test = gbdt_cat_ADHD.predict_proba(X_test_final_cat)[:, 1]
pred_adhd_2_test = gbdt_quant_ADHD.predict_proba(X_test_final_quant)[:, 1]

pred_gender_1_test = gbdt_cat_female.predict_proba(X_test_final_cat)[:, 1]
pred_gender_2_test = gbdt_quant_female.predict_proba(X_test_final_quant)[:, 1]

# Stack predictions as new features
meta_X_test_ADHD = np.column_stack((pred_adhd_1_test, pred_adhd_2_test))
meta_X_test_gender = np.column_stack((pred_gender_1_test, pred_gender_2_test))

# Meta-model predictions
final_pred_adhd = meta_model_adhd.predict(meta_X_test_ADHD)
final_pred_gender = meta_model_gender.predict(meta_X_test_gender)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(


In [ ]:
print(y_test.shape)
print(final_pred_gender.shape)

(243,)
(304,)


In [ ]:
# Accuracy for ADHD prediction
accuracy_adhd = accuracy_score(y_test_ADHD, final_pred_adhd[:194])
print(f"Accuracy for ADHD prediction: {accuracy_adhd}")

# F1 Score for ADHD prediction
f1_adhd = f1_score(y_test_ADHD, final_pred_adhd[:194], average='weighted')
print(f"F1 Score for ADHD prediction: {f1_adhd}")

# Accuracy for Gender prediction
accuracy_gender = accuracy_score(y_test, final_pred_gender[:243])
print(f"Accuracy for Gender prediction: {accuracy_gender}")

# F1 Score for Gender prediction
f1_gender = f1_score(y_test, final_pred_gender[:243], average='weighted')
print(f"F1 Score for Gender prediction: {f1_gender}")

Accuracy for ADHD prediction: 0.30927835051546393
F1 Score for ADHD prediction: 0.14611575614903807
Accuracy for Gender prediction: 0.691358024691358
F1 Score for Gender prediction: 0.5651978012075335


In [ ]:
participant_ids = test_quantitative['participant_id']

# Create a dictionary to hold the data
data = {
    'participant_id': participant_ids,
    'ADHD_Outcome': final_pred_adhd,
    'Sex_F': final_pred_gender
}

# Create a Pandas DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('predictions.csv', index=False)

# Print the first few rows to verify (optional)
print(df.head(30))


   participant_id  ADHD_Outcome  Sex_F
0    Cfwaf5FX7jWK             0      0
1    vhGrzmvA3Hjq             0      0
2    ULliyEXjy4OV             0      0
3    LZfeAb1xMtql             0      0
4    EnFOUv0YK1RG             0      0
5    3VbkvJ22j9Fu             0      0
6    PRKZcnOgqcuk             0      0
7    DuVUuyMZi5qV             0      0
8    uM4etVLZrgMg             0      0
9    BpzyExrET5ta             0      0
10   sAqeb6F4lz97             0      0
11   u7XOOvHirIx7             0      0
12   aEPm4bEQvbYi             0      0
13   Fj9A5PWsIWKT             0      0
14   19mb5yGJigtw             0      0
15   v1nMpCoLGU0V             0      0
16   hRPuz4zpsEbw             0      0
17   mT8A6xa1O4Ro             0      0
18   4QBTjDoVpVt6             0      0
19   0X2H4LroxZcw             0      0
20   9CH7UxXuznUa             0      0
21   nU73zzjTnr4A             0      0
22   uEZHGukIUQ0k             0      0
23   jCzQwkpfgZyQ             0      0
24   Ljvrs76QJuI5        

Predicting everything is 0 ?

## Using only quant ADHD model and categorical gender model

In [ ]:
from sklearn.model_selection import train_test_split

tr_categorical = pd.read_excel("/content/TRAIN_CATEGORICAL_METADATA_new.xlsx")
tr_quantitative = pd.read_excel("/content/TRAIN_QUANTITATIVE_METADATA_new.xlsx")
y_tr = pd.read_excel("/content/TRAINING_SOLUTIONS.xlsx")


tr_quantitative = tr_quantitative.drop(['participant_id', 'MRI_Track_Age_at_Scan'], axis=1)
tr_quantitative = tr_quantitative.apply(lambda col: col.fillna(col.mean()), axis=0)

tr_categorical = tr_categorical.drop(['participant_id'], axis=1)
tr_categorical = tr_categorical.apply(lambda col: col.fillna(col.mode()[0]), axis=0)

# Split into training and test sets (80% train, 20% test)
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(tr_categorical, y_tr, test_size=0.2, random_state=42)
X_train_q, X_test_q, y_train_q, y_test_q = train_test_split(tr_quantitative, y_tr, test_size=0.2, random_state=42)

# Print dataset sizes
print(f"Training set size: {X_train_c.shape}, {X_train_q.shape}")
print(f"Test set size: {X_test_c.shape}, {X_test_q.shape}")


Training set size: (970, 9), (970, 17)
Test set size: (243, 9), (243, 17)


In [ ]:
tr_pred_adhd = gbdt_quant_ADHD.predict(X_train_q)
tr_pred_gender = gbdt_cat_female.predict(X_train_c)

# Accuracy for ADHD prediction
accuracy_adhd = accuracy_score(y_train_q['ADHD_Outcome'], tr_pred_adhd)
print(f"Accuracy for ADHD prediction: {accuracy_adhd}")

# F1 Score for ADHD prediction
f1_adhd = f1_score(y_train_q['ADHD_Outcome'], tr_pred_adhd, average='weighted')
print(f"F1 Score for ADHD prediction: {f1_adhd}")

# Accuracy for Gender prediction
accuracy_gender = accuracy_score(y_train_c['Sex_F'], tr_pred_gender)
print(f"Accuracy for Gender prediction: {accuracy_gender}")

# F1 Score for Gender prediction
f1_gender = f1_score(y_train_c['Sex_F'], tr_pred_gender, average='weighted')
print(f"F1 Score for Gender prediction: {f1_gender}")

Accuracy for ADHD prediction: 0.6731958762886598
F1 Score for ADHD prediction: 0.5417090661940788
Accuracy for Gender prediction: 0.5484536082474227
F1 Score for Gender prediction: 0.5381145190699501


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(


In [ ]:
#ON PARITIONED TEST DATA
test_pred_adhd = gbdt_quant_ADHD.predict(X_test_q)
test_pred_gender = gbdt_cat_female.predict(X_test_c)

# Accuracy for ADHD prediction
accuracy_adhd = accuracy_score(y_test_q['ADHD_Outcome'], test_pred_adhd)
print(f"Accuracy for ADHD prediction: {accuracy_adhd}")

# F1 Score for ADHD prediction
f1_adhd = f1_score(y_test_q['ADHD_Outcome'], test_pred_adhd, average='weighted')
print(f"F1 Score for ADHD prediction: {f1_adhd}")

# Accuracy for Gender prediction
accuracy_gender = accuracy_score(y_test_c['Sex_F'], test_pred_gender)
print(f"Accuracy for Gender prediction: {accuracy_gender}")

# F1 Score for Gender prediction
f1_gender = f1_score(y_test_c['Sex_F'], test_pred_gender, average='weighted')
print(f"F1 Score for Gender prediction: {f1_gender}")


Accuracy for ADHD prediction: 0.7325102880658436
F1 Score for ADHD prediction: 0.6194148754190982
Accuracy for Gender prediction: 0.6419753086419753
F1 Score for Gender prediction: 0.6346312124089901


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(


In [ ]:
participant_ids = test_quantitative['participant_id']

# Create a dictionary to hold the data
data = {
    'ADHD_Outcome': test_pred_adhd,
    'Sex_F': test_pred_gender
}

# Create a Pandas DataFrame
df = pd.DataFrame(data)

# Print the first few rows to verify (optional)
print(df.head(30))

    ADHD_Outcome  Sex_F
0              1      0
1              1      0
2              1      0
3              1      1
4              1      0
5              1      1
6              1      0
7              1      0
8              1      0
9              1      0
10             1      0
11             1      1
12             1      0
13             1      0
14             1      1
15             1      0
16             1      0
17             1      0
18             1      0
19             1      0
20             1      0
21             1      0
22             1      0
23             1      0
24             1      0
25             1      0
26             1      1
27             1      0
28             1      0
29             1      0


On Kaggle test data to make prediction

In [ ]:
#ON TEST DATA
#Quant ADHD and cat female
test_categorical = pd.read_excel("/content/TEST_CATEGORICAL.xlsx")
test_quantitative = pd.read_excel("/content/TEST_QUANTITATIVE_METADATA.xlsx")

X_test_final_quant = test_quantitative.drop(['participant_id', 'MRI_Track_Age_at_Scan'], axis=1)
X_test_final_quant = X_test_final_quant.apply(lambda col: col.fillna(col.mean()), axis=0)

X_test_final_cat = test_categorical.drop(['participant_id'], axis=1)
X_test_final_cat = X_test_final_cat.apply(lambda col: col.fillna(col.mode()[0]), axis=0)

# Get base model predictions on test data
final_pred_adhd = gbdt_quant_ADHD.predict(X_test_final_quant)

final_pred_gender = gbdt_cat_female.predict(X_test_final_cat)


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(


In [ ]:
participant_ids = test_quantitative['participant_id']

# Create a dictionary to hold the data
data = {
    'participant_id': participant_ids,
    'ADHD_Outcome': final_pred_adhd,
    'Sex_F': final_pred_gender
}

# Create a Pandas DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('pred_ensemble.csv', index=False)

# Print the first few rows to verify (optional)
print(df.head(30))

   participant_id  ADHD_Outcome  Sex_F
0    Cfwaf5FX7jWK             1      1
1    vhGrzmvA3Hjq             1      1
2    ULliyEXjy4OV             1      0
3    LZfeAb1xMtql             1      1
4    EnFOUv0YK1RG             1      0
5    3VbkvJ22j9Fu             1      0
6    PRKZcnOgqcuk             1      1
7    DuVUuyMZi5qV             1      0
8    uM4etVLZrgMg             1      0
9    BpzyExrET5ta             1      1
10   sAqeb6F4lz97             1      0
11   u7XOOvHirIx7             1      0
12   aEPm4bEQvbYi             1      0
13   Fj9A5PWsIWKT             1      0
14   19mb5yGJigtw             1      0
15   v1nMpCoLGU0V             1      0
16   hRPuz4zpsEbw             1      1
17   mT8A6xa1O4Ro             1      0
18   4QBTjDoVpVt6             1      1
19   0X2H4LroxZcw             1      0
20   9CH7UxXuznUa             1      0
21   nU73zzjTnr4A             1      0
22   uEZHGukIUQ0k             1      0
23   jCzQwkpfgZyQ             1      0
24   Ljvrs76QJuI5        

In [ ]:
print(df.shape)
print(df['ADHD_Outcome'].sum())
print(df['Sex_F'].sum())

(304, 3)
304
54
